In [26]:
import google.generativeai as genai
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
import spacy
import json
import re
import time
import logging
import asyncio
from gpt4all import GPT4All
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from dataclasses import dataclass, asdict
from typing import List, Dict, Optional, Tuple, Any
from concurrent.futures import ThreadPoolExecutor
import ahocorasick
from collections import defaultdict, deque, Counter
from itertools import combinations
import pandas as pd

In [27]:
class GeminiEnhancedTextProcessor:
    def __init__(self, gemini_api_keys: List[str], model_name: str = "gemini-2.0-flash"):
        self.gemini_api_keys = gemini_api_keys
        self.current_key_index = 0
        self.key_usage_count = {key: 0 for key in gemini_api_keys}
        self.key_last_used = {key: 0 for key in gemini_api_keys}
        self.model_name = model_name
        self._setup_gemini_client()
        try:
            self.nlp = spacy.load("ru_core_news_sm")
        except OSError:
            raise RuntimeError("Требуется установка: python -m spacy download ru_core_news_sm")
        self.sentence_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
        self.formula_patterns = [
            r"[yY]['’ʹ′‵‶‶‷]?\s*=\s*f[ₙ₀-₉]*\s*\([^()]*(?:\([^()]*(?:\([^()]*(?:\([^()]*\)[^()]*)*\)[^()]*)*\)[^()]*)*\)(?:[WB][₀-₉ₙₖ]*\s*[+\-]\s*[WB][₀-₉ₖ]*)*", 
            r"[EεΣ]\s*=\s*\([^)]+(?:\([^)]*\)[^)]*)*\)\s*/\s*[nmδ]", 
            r"h_[t]\s*=\s*f\([^)]+\)", 
            r"[εδ²]+\s*[<≤]\s*(?:\[[^\]]+\]|[^,\n.; ]+)\s*/\s*[m]", 
            r"\|[^|]+\|\s*[≈=]\s*2\^\{?[^}\n.,;]+\}?", 
            r"(?:\b[a-zA-Zα-ωΑ-Ωεδ]['’ʹ′‵‶‶‷]*(?:[₀-₉ₙₖₜᵢⱼₓ]|['’ʹ′‵‶‶‷]\s*)*\b(?:\[[^\]]+\])?|\b[a-zA-Zα-ωΑ-Ωεδ][₀-₉ₙₖₜᵢⱼₓ]*\s*\([^)]+\)|\([^)]+\))\s*[=≈<>≤≥]\s*(?:[^,\n.; ]|FORMULA_\d+__){1,150}",
            r"\b[a-zA-Zα-ωΑ-Ωεδ][₀-₉ₙₖₜᵢⱼₓ]*\s*\([^)]+\)",
            r"\b[WB][₀-₉ₙₖ]*\s*[+\-*/]\s*[WBηДЕЛа-яА-Яα-ωΑ-Ωεδ][₀-₉ₙₖ]*", 
            r"\b[a-zA-Zα-ωΑ-Ωεδ]['’ʹ′‵‶‶‷]*(?:[₀-₉ₙₖₜᵢⱼₓ]|['’ʹ′‵‶‶‷]\s*)+(?:\[[^\]]+\])?\b", 
            r"\([a-zA-Zα-ωΑ-Ωεδ]['’ʹ′‵‶‶‷]*[₀-₉ₙₖₜᵢⱼₓ]*\)",
            r"\b[a-zA-Zα-ωΑ-Ωεδ]\s*['’ʹ′‵‶‶‷]\s*-\s*[a-zA-Zα-ωΑ-Ωεδ]['’ʹ′‵‶‶‷]*\b", 
            r"\b\d+\^[a-zA-Zα-ωΑ-Ωεδ(][^)\s]*\)?", 
            r"\b[a-zA-Z0-9₀-₉ₙₖₜᵢⱼₓ chronically]+\s*(?:×|\*)\s*[a-zA-Z0-9₀-₉ₙₖₜᵢⱼₓ chronically]+\b", 
            r"\b[I]\([^)]+;[^)]+\)\s*/\s*[I]\([^)]+;[^)]+\)",
            r"\b[I]\s*\([^)]+;\s*[^)]+\)\s*=\s*(?:[^,\n.;()]|\([^)]*\)|__MATH_FORMULA_\d+__)+",
        ]
        self.symbol_normalize = {'—': '-', '–': '-', '«': '"', '»': '"'}
        self.protected_terms = {
            'weights', 'biases', 'loss function', 'gradient descent',
            'neural network', 'activation function', 'backpropagation',
            'dropout', 'batch normalization', 'transformer', 'attention',
            'deep learning', 'machine learning', 'cross-validation'
        }
        self.term_synonyms = {}
        self.abbreviations = {}
        self.context_rules = {}
        self.ac_automaton = None
        self.automation_pairs = []
        self.chunk_size = 2000
        self.rate_limit_delay = 1.0
        self._gemini_cache = {}
        self.processed_segments = set()
        self.formula_counter = 0
        self.formula_names = {}
        
    def _setup_gemini_client(self):
        current_key = self.gemini_api_keys[self.current_key_index]
        genai.configure(api_key=current_key)
        self.gemini_model = genai.GenerativeModel(self.model_name)

    def _rotate_api_key(self):
        self.current_key_index = (self.current_key_index + 1) % len(self.gemini_api_keys)
        self._setup_gemini_client()
        print(f"Переключился на API ключ #{self.current_key_index + 1}")

    def smart_text_chunking(self, text: str) -> List[str]:
        doc = self.nlp(text)
        sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()] 
        chunks = []
        current_chunk = ""
        for sentence in sentences:
            if len(current_chunk) + len(sentence) + 1 < self.chunk_size:
                current_chunk += (" " if current_chunk else "") + sentence
            else:
                if current_chunk:
                    chunks.append(current_chunk.strip())
                current_chunk = sentence
        if current_chunk.strip():
            chunks.append(current_chunk.strip())
        return chunks
    
    def _extract_and_protect_formulas(self, text: str) -> Tuple[str, List[str], Dict[str, str]]:
        processed_text = re.sub(r'подчеркивание+[A-Za-z]*|FORMU[а-я]*A[₀-₉]*', '', text)
        all_matches = []
        combined_pattern = '|'.join(f'({pattern})' for pattern in self.formula_patterns)
        for match in re.finditer(combined_pattern, processed_text, re.IGNORECASE | re.UNICODE):
            formula = match.group().strip()
            if self._is_valid_formula(formula):
                all_matches.append((match.start(), match.end(), formula))
        if not all_matches:
            return processed_text, [], {}
        all_matches.sort(key=lambda x: (x[0], -(x[1] - x[0])))
        final_matches = []
        last_end = -1
        for start, end, formula in all_matches:
            if start >= last_end:  
                final_matches.append((start, end, formula))
                last_end = end
        final_matches.sort(key=lambda x: x[0], reverse=True)
        formulas = []
        formula_placeholders = {}
        for start, end, formula in final_matches:
            self.formula_counter += 1
            placeholder = f"__MATH_FORMULA_{self.formula_counter}__"
            formula_dict = {
                'placeholder': placeholder,
                'formula': formula
            }
            formulas.append(formula_dict)
            formula_placeholders[placeholder] = formula
            processed_text = processed_text[:start] + placeholder + processed_text[end:]
        return processed_text, formulas[::-1], formula_placeholders
    
    def _is_valid_formula(self, text: str) -> bool:
        """Проверка валидности математической формулы."""
        if len(text) < 2:
            return False
        math_indicators = [
            r'[=≈<>≤≥]',  
            r'[₀-₉ₙₖₜ]',  
            r'[∑∏∫∂∇]',   
            r'f[₀-₉ₙ]*\s*\(', 
            r'[+\-*/^]',   
            r'[α-ωΑ-Ωεδ]', 
            r'[WBhxy]_[₀-₉ₙₖₜ]', 
            r'\^[₀-₉{}\[\]]', 
            r'\|[^|]+\|',
            r'[₀-₁₀ₙₖₜᵢⱼₓ chronically]', 
            r"['’ʹ′‵‶‶‷]",        
            r'[a-zA-Zα-ωΑ-Ωεδ]\s*\(', 
            r'[WBXYHV]\s*_[^_\s]+', 
            r'\^[^{}\[\]\s]+|\^\{[^{}]*\}|\^\[[^\]]*\]', 
            r'[⇒→∈∀∃]',      
            r'\d+\.\d+',      
            r'log|exp|sin|cos|tan|det|tr'   
        ]
        matches = sum(1 for pattern in math_indicators if re.search(pattern, text))
        return matches >= 2

    def _normalize_text_content(self, text: str) -> str:
        text = re.sub(r'[₀-₉ₙₖₜ]+(?![A-Za-z])', '', text)  
        text = re.sub(r'[α-ωΑ-Ωεδ](?![A-Za-z])', '', text)  
        text = re.sub(r'[∑∏∫∂∇≈≤≥]', '', text) 
        for old, new in self.symbol_normalize.items():
            text = text.replace(old, new)
        return text

    def _post_process_text(self, text: str) -> str:
        """Постобработка: убираем тире и приводим к нижнему регистру"""
        formula_pattern = r'__MATH_FORMULA_\d+__'
        formulas_found = re.findall(formula_pattern, text)
        temp_text = re.sub(formula_pattern, '<<<FORMULA>>>', text)
        temp_text = temp_text.replace('-', ' ')
        temp_text = temp_text.lower()
        for formula in formulas_found:
            temp_text = temp_text.replace('<<<formula>>>', formula, 1)
        return temp_text

    def _fix_word_boundaries(self, text: str) -> str:
        text = re.sub(r'([а-яё])([А-ЯЁ])', r'\1 \2', text) 
        text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)    
        text = re.sub(r'([а-яёa-z])([А-ЯЁA-Z][а-яёa-z])', r'\1 \2', text)  
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\s+([,.!?;:])', r'\1', text)
        text = re.sub(r'([.!?])\s*([А-ЯЁA-Z])', r'\1 \2', text)
        text = re.sub(r'([а-яёa-z])([,.!?;:])([а-яёa-z])', r'\1\2 \3', text)
        text = re.sub(r'([а-яёa-z]{2,})([.!?])([А-ЯЁA-Z][а-яёa-z])', r'\1\2 \3', text) 
        return text.strip()

    def process_text_segment(self, text: str) -> str:
        if not text or not text.strip():
            return text
        segment_hash = hash(text.strip())
        if segment_hash in self.processed_segments:
            return text
        text = self._normalize_text_content(text)
        if not (self.ac_automaton and self.automation_pairs):
            text = self._fix_word_boundaries(text)
            self.processed_segments.add(segment_hash)
            return text
        valid_matches = []
        text_len = len(text) 
        try:
            for end_index, pair_index in self.ac_automaton.iter(text):
                if not (0 <= pair_index < len(self.automation_pairs)):
                    continue
                original, replacement = self.automation_pairs[pair_index]
                original_len = len(original)
                start_index = end_index - original_len + 1
                if (0 <= start_index < text_len and 
                    end_index < text_len and
                    text[start_index:end_index + 1] == original):
                    if (self._safe_to_replace(text, start_index, end_index + 1, original) and
                        self._is_word_boundary(text, start_index, end_index + 1)):
                        valid_matches.append((start_index, end_index + 1, replacement))
        except Exception:
            valid_matches = []
        if valid_matches:
            valid_matches.sort(key=lambda x: x[0], reverse=True)
            for start, end, replacement in valid_matches:
                text = text[:start] + replacement + text[end:]
        text = self._fix_word_boundaries(text)
        self.processed_segments.add(segment_hash)
        return text

    def _safe_to_replace(self, text: str, start: int, end: int, original: str) -> bool:
        context = text[max(0, start-10):min(len(text), end+10)].lower()
        return not any(term in context for term in self.protected_terms)

    def _is_word_boundary(self, text: str, start: int, end: int) -> bool:
        before_ok = start == 0 or not text[start-1].isalnum()
        after_ok = end >= len(text) or not text[end].isalnum()
        return before_ok and after_ok

    def build_automaton(self):
        self.ac_automaton = ahocorasick.Automaton()
        replacement_dict = {}
        for source in [self.abbreviations, self.term_synonyms]:
            for key, value in source.items():
                if not self._is_protected_term(str(key)):
                    if isinstance(value, list):
                        replacement_dict[key] = ' '.join(value)
                    else:
                        replacement_dict[key] = str(value)
        self.automation_pairs = []
        for idx, (term, replacement) in enumerate(replacement_dict.items()):
            term_str = str(term).strip()
            if len(term_str) > 1 and term_str != replacement:
                try:
                    self.ac_automaton.add_word(term_str, idx)
                    self.automation_pairs.append((term_str, replacement))
                except Exception:
                    continue
        if self.automation_pairs:
            self.ac_automaton.make_automaton()

    def _is_protected_term(self, term: str) -> bool:
        term_lower = term.lower()
        return any(protected in term_lower or term_lower in protected 
                  for protected in self.protected_terms)

    async def _call_gemini_with_retry(self, prompt: str, max_retries: int = 3) -> Dict:
        for attempt in range(max_retries):
            try:
                current_key = self.gemini_api_keys[self.current_key_index]
                self.key_usage_count[current_key] += 1
                self.key_last_used[current_key] = time.time()
                response = self.gemini_model.generate_content(prompt)
                json_text = response.text.strip()
                json_match = re.search(r'\{.*\}', json_text, re.DOTALL)
                if json_match:
                    return json.loads(json_match.group())
                else:
                    return {}         
            except Exception as e:
                error_str = str(e).lower()
                if 'quota' in error_str or 'limit' in error_str or 'rate' in error_str:
                    print(f"Лимит API ключа #{self.current_key_index + 1}, переключаемся...")
                    self._rotate_api_key()
                    await asyncio.sleep(2)
                elif attempt < max_retries - 1:
                    await asyncio.sleep(2 ** attempt)
                else:
                    print(f"Ошибка Gemini после {max_retries} попыток: {e}")
                    return {}
        return {}

    def create_analysis_prompt(self, text_chunk: str) -> str:
        return (
            f"Проанализируй научный текст по ML/нейросетям и верни JSON:\n\n"
            f"ВАЖНО: НЕ разбивай термины 'weights', 'biases', 'loss function'!\n\n"
            f"Найди ВСЕ математические формулы, уравнения, выражения включая:\n"
            f"- Функции: f(x), f_n(...), h_t = f(...)\n"
            f"- Уравнения: E = ..., I(...) = ..., y' = ...\n"
            f"- Неравенства: ε² < [...], |T| ≈ 2^5\n"
            f"- Выражения со спецсимволами: Σ, ∏, ∂, ∇, ≈, ≤, ≥\n\n"
            f"Текст: {text_chunk}\n\n"
            f"JSON формат:\n{{\n"
            f'  "formulas": ["полная формула 1", "полная формула 2"],\n'
            f'  "formula_patterns": ["паттерн1", "паттерн2"],\n'
            f'  "abbreviations": {{"аббр": "расшифровка"}},\n'
            f'  "synonyms": {{"термин": ["синоним1", "синоним2"]}}\n'
            f"}}"
        )

    async def process_with_gemini(self, chunks: List[str]) -> Dict:
        results = {"formulas": set(), "formula_patterns": set(), "abbreviations": {}, "synonyms": {}}
        for i, chunk in enumerate(chunks):
            cache_key = f"unified_{hash(chunk[:100])}"
            if cache_key in self._gemini_cache:
                result = self._gemini_cache[cache_key]
            else:
                prompt = self.create_analysis_prompt(chunk)
                result = await self._call_gemini_with_retry(prompt)
                self._gemini_cache[cache_key] = result
            self._merge_results(results, result)
            if i < len(chunks) - 1:
                await asyncio.sleep(self.rate_limit_delay)
        return results

    def _merge_results(self, target: Dict, source: Dict):
        if "formulas" in source:
            if isinstance(source["formulas"], (list, set)):
                target["formulas"].update(source["formulas"])
        if "formula_patterns" in source:
            if isinstance(source["formula_patterns"], (list, set)):
                target["formula_patterns"].update(source["formula_patterns"])
        for key in ["abbreviations", "synonyms"]:
            if key in source and isinstance(source[key], dict):
                filtered = {k: v for k, v in source[key].items() 
                           if not self._is_protected_term(str(k))}
                target[key].update(filtered)

    def apply_enhancements(self, enhancements: Dict):
        if "formulas" in enhancements:
            for formula in enhancements["formulas"]:
                if formula and len(str(formula).strip()) > 3:
                    escaped_formula = re.escape(str(formula).strip())
                    if escaped_formula not in self.formula_patterns:
                        self.formula_patterns.append(escaped_formula)
        if "formula_patterns" in enhancements:
            for pattern in enhancements["formula_patterns"]:
                if pattern and pattern not in self.formula_patterns:
                    try:
                        re.compile(pattern)
                        self.formula_patterns.append(pattern)
                    except re.error:
                        continue
        self.abbreviations.update(enhancements.get("abbreviations", {}))
        self.term_synonyms.update(enhancements.get("synonyms", {}))

    def _thematic_segmentation(self, sentences: List[str]) -> List[List[str]]:
        if len(sentences) < 2:
            return [sentences]
        try:
            embeddings = self.sentence_model.encode(sentences)
            similarities = cosine_similarity(embeddings[:-1], embeddings[1:])
            segments = []
            current_segment = [sentences[0]]
            threshold = max(0.3, np.percentile(similarities.diagonal(), 40))
            for i, sim in enumerate(similarities.diagonal()):
                if sim > threshold:
                    current_segment.append(sentences[i + 1])
                else:
                    segments.append(current_segment)
                    current_segment = [sentences[i + 1]]
            segments.append(current_segment)
            return segments
        except Exception:
            return [sentences[i:i+4] for i in range(0, len(sentences), 4)]

    async def process_text_enhanced(self, text: str, use_gemini: bool = True, n_workers: int = 2) -> Dict:
        try:
            self.processed_segments.clear()
            word_freq = Counter(re.findall(r'\b\w+\b', text.lower()))
            if use_gemini:
                chunks = self.smart_text_chunking(text)
                gemini_results = await self.process_with_gemini(chunks)
                self.apply_enhancements(gemini_results)
            text_no_formulas, formulas, formula_placeholders = self._extract_and_protect_formulas(text)
            self.build_automaton()
            chunks = [text_no_formulas[i:i+1000] for i in range(0, len(text_no_formulas), 1000)]
            with ThreadPoolExecutor(max_workers=min(n_workers, len(chunks))) as executor:
                processed_chunks = list(executor.map(self.process_text_segment, chunks))
            processed_text = ''.join(processed_chunks)
            processed_text = self._fix_word_boundaries(processed_text)
            processed_text = self._post_process_text(processed_text)
            doc = self.nlp(processed_text)
            sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]
            unique_sentences = []
            seen = set()
            for sentence in sentences:
                clean = re.sub(r'\s+', ' ', sentence.lower().strip())
                if clean not in seen and len(clean) > 5:
                    unique_sentences.append(sentence)
                    seen.add(clean)
            segments = self._thematic_segmentation(unique_sentences)
            return {
                'processed_text': ' '.join(unique_sentences),
                'formulas': formulas,
                'formula_placeholders': formula_placeholders,  
                'thematic_segments': segments,
                'statistics': {
                    'words_top': dict(word_freq.most_common(20)),
                    'sentences_original': len(sentences),
                    'sentences_unique': len(unique_sentences),
                    'formulas_found': len(formulas),
                    'gemini_formulas': len(gemini_results.get('formulas', [])) if use_gemini else 0,
                    'total_patterns': len(self.formula_patterns),
                    'protected_terms_count': len([t for t in self.protected_terms 
                                                if t in processed_text.lower()])
                },
                'gemini_analysis': {
                    'found_formulas': list(gemini_results.get('formulas', [])) if use_gemini else [],
                    'found_patterns': list(gemini_results.get('formula_patterns', [])) if use_gemini else [],
                    'abbreviations': gemini_results.get('abbreviations', {}) if use_gemini else {},
                    'synonyms': gemini_results.get('synonyms', {}) if use_gemini else {}
                } if use_gemini else {},
                'api_usage': dict(self.key_usage_count)
            }
        except Exception as e:
            raise RuntimeError(f"Ошибка обработки: {str(e)}")

In [36]:
logger = logging.getLogger(__name__)

@dataclass
class Entity:
    id: str
    name: str
    type: str
    description: str = ""
    source_chunks: List[int] = None  
    
    def __post_init__(self):
        if self.source_chunks is None:
            self.source_chunks = []

@dataclass  
class Relation:
    source: str
    target: str
    type: str
    description: str = ""
    source_chunks: List[int] = None  

    def __post_init__(self):
        if self.source_chunks is None:
            self.source_chunks = []

class ImprovedKnowledgeExtractor:
    def __init__(self, 
                 gemini_api_keys: List[str], 
                 generation_model_path: str, 
                 model_name: str = "gemini-2.0-flash", 
                 chunk_size: int = 5, 
                 max_sentences: int = 200,
                 fallback_timeout: int = 30):
        """
        Инициализация экстрактора знаний
        
        Args:
            gemini_api_keys: Список API ключей для Gemini
            generation_model_path: Путь к локальной модели
            model_name: Название модели Gemini
            chunk_size: Размер чанка в предложениях
            max_sentences: Максимальное количество предложений для обработки
            fallback_timeout: Таймаут для переключения на локальную LLM (секунды)
        """
        self.gemini_api_keys = gemini_api_keys
        self.generation_model_path = generation_model_path
        self.model_name = model_name
        self.chunk_size = chunk_size
        self.max_sentences = max_sentences
        self.fallback_timeout = fallback_timeout
        self._init_local_llm()
        self.current_key_index = 0
        self.key_usage_count = {key: 0 for key in gemini_api_keys}
        self.key_last_used = {key: 0 for key in gemini_api_keys}
        self.gemini_available = True
        self._setup_gemini_client()
        self.generation_config = genai.types.GenerationConfig(
            temperature=0.1,  
            top_p=0.2,
            top_k=3,
            max_output_tokens=2000,  
            candidate_count=1
        )
        self.formula_pattern = re.compile(r'__MATH_FORMULA_(\d+)__')
        self._setup_prompts()
        self._init_nltk()
        
    def _init_local_llm(self):
        """Инициализация локальной LLM"""
        try:
            self.local_llm = GPT4All(self.generation_model_path)
            logger.info("Локальная LLM успешно инициализирована")
        except Exception as e:
            logger.error(f"Ошибка инициализации локальной LLM: {e}")
            self.local_llm = None

    def _init_nltk(self):
        """Инициализация NLTK для разделения предложений"""
        try:
            nltk.data.find('tokenizers/punkt')
        except LookupError:
            logger.info("Скачиваем punkt tokenizer для NLTK...")
            nltk.download('punkt')

    def _setup_gemini_client(self):
        """Настройка клиента Gemini"""
        try:
            current_key = self.gemini_api_keys[self.current_key_index]
            genai.configure(api_key=current_key)
            self.gemini_model = genai.GenerativeModel(self.model_name)
            logger.info(f"Gemini клиент настроен с ключом #{self.current_key_index + 1}")
        except Exception as e:
            logger.error(f"Ошибка настройки Gemini клиента: {e}")
            self.gemini_available = False

    def _rotate_api_key(self):
        """Ротация API ключей"""
        if len(self.gemini_api_keys) > 1:
            self.current_key_index = (self.current_key_index + 1) % len(self.gemini_api_keys)
            self._setup_gemini_client()
            logger.info(f"Переключился на API ключ #{self.current_key_index + 1}")
        else:
            logger.warning("Только один API ключ доступен")

    def _setup_prompts(self):
        """Настройка промптов для извлечения"""
        self.entity_prompt = """Внимательно проанализируй текст и найди все важные понятия, концепции, методы, алгоритмы, теории и другие объекты.

ВНИМАНИЕ: В тексте могут встречаться заменители формул вида __MATH_FORMULA_N__, где N - номер формулы. Учитывай их при анализе как математические выражения.

СТРОГО следуй формату ответа:
```
name: точное название объекта/концепта без скобок, короткое но информативное
type: тип (tasks/methods/algorithms/data/theories/concepts/formulas/functions/entities/processes/mechanisms/systems/phenomena/tools/techniques/models/structures)
description: краткое но информативное описание
---
```

Обязательно найди минимум 3-5 объектов в тексте. Не оставляй ответ пустым.

Текст для анализа:
{text}

Извлеченные объекты:"""

        self.relation_prompt = """Найди ВСЕ связи между данными объектами в тексте. Ищи как прямые, так и косвенные связи.

ВНИМАНИЕ: В тексте могут встречаться заменители формул вида __MATH_FORMULA_N__. Учитывай их при поиске связей.

Известные объекты: {entities}

СТРОГО следуй формату ответа:
```
source: точное название первого объекта
target: точное название второго объекта
type: тип связи (defines/subclass-of/instance-of/superclass-of/equals/same-as/connected-to/interacts-with/part-of/used-for/enables/causes/prevents/contains/requires/produces/controls/creates/generates/before/after/during/co-occurs-with/represents/implements/applies/based-on/derived-from/similar-to)
description: подробное описание связи
---
```

Найди минимум 2-3 связи, если объекты упоминаются в тексте.

Текст для анализа связей:
{text}

Найденные связи:"""

    # def split_text_into_sentences(self, text: str) -> List[str]:
    #     """
    #     Автоматическое разделение текста на предложения с обработкой формул
        
    #     Args:
    #         text: Входной текст
            
    #     Returns:
    #         Список предложений
    #     """
    #     formula_placeholders = {}
    #     formula_matches = list(self.formula_pattern.finditer(text))
        
    #     for match in formula_matches:
    #         placeholder = f"FORMULA_PLACEHOLDER_{match.group(1)}"
    #         formula_placeholders[placeholder] = match.group(0)
    #         text = text.replace(match.group(0), placeholder)
    #     try:
    #         sentences = sent_tokenize(text, language='russian')
    #     except:
    #         sentences = [s.strip() for s in text.split('.') if s.strip()]
    #     processed_sentences = []
    #     for sentence in sentences:
    #         for placeholder, original in formula_placeholders.items():
    #             sentence = sentence.replace(placeholder, original)
    #         if len(sentence.strip()) > 10:
    #             processed_sentences.append(sentence.strip())
    #     logger.info(f"Текст разделен на {len(processed_sentences)} предложений")
    #     return processed_sentences

    def split_text_into_sentences(self, text: str) -> List[str]:
        if not text or not text.strip():
            return []
        formula_placeholders = {}
        protected_text = text
        formula_matches = list(self.formula_pattern.finditer(text))
        if formula_matches:
            formula_matches.sort(key=lambda m: m.start(), reverse=True)  
            for match in formula_matches:
                placeholder = f"FORMULA_PLACEHOLDER_{match.group(1)}"
                formula_placeholders[placeholder] = match.group(0)
                start, end = match.span()
                protected_text = protected_text[:start] + placeholder + protected_text[end:]
        try:
            sentences = sent_tokenize(protected_text, language='russian')
        except Exception:
            sentences = [s.strip() for s in protected_text.split('.') if s.strip()]
        if not formula_placeholders:
            processed_sentences = [s.strip() for s in sentences if len(s.strip()) > 10]
            logger.info(f"Текст разделен на {len(processed_sentences)} предложений")
            return processed_sentences
        placeholder_pattern = '|'.join(re.escape(placeholder) for placeholder in formula_placeholders.keys())
        def replace_placeholder(match):
            return formula_placeholders[match.group(0)]
        processed_sentences = []
        for sentence in sentences:
            if len(sentence.strip()) <= 10:
                continue
            if any(placeholder in sentence for placeholder in formula_placeholders):
                restored_sentence = re.sub(placeholder_pattern, replace_placeholder, sentence)
            else:
                restored_sentence = sentence
            processed_sentences.append(restored_sentence.strip())
        logger.info(f"Текст разделен на {len(processed_sentences)} предложений")
        return processed_sentences

    def _preprocess_text_chunk(self, chunk_text: str) -> str:
        """
        Предобработка текста чанка
        
        Args:
            chunk_text: Текст чанка
            
        Returns:
            Обработанный текст
        """
        def replace_formula(match):
            formula_num = match.group(1)
            return f"математическая формула номер {formula_num}"
        processed_text = self.formula_pattern.sub(replace_formula, chunk_text)
        processed_text = re.sub(r'\s+', ' ', processed_text)
        processed_text = processed_text.strip()
        return processed_text

    def _create_chunks(self, sentences: List[str]) -> List[Dict]:
        """
        Создает чанки из предложений с оптимизацией для больших текстов
        
        Args:
            sentences: Список предложений
            
        Returns:
            Список чанков
        """
        chunks = []
        for i in range(0, len(sentences), self.chunk_size):
            chunk_sentences = sentences[i:i + self.chunk_size]
            chunk_text = ' '.join(chunk_sentences)
            processed_text = self._preprocess_text_chunk(chunk_text)
            if len(processed_text) > 2000:
                processed_text = processed_text[:2000] + "..."
            chunks.append({
                'id': i // self.chunk_size,
                'text': processed_text,
                'original_text': chunk_text,
                'sentences': chunk_sentences,
                'start_idx': i,
                'end_idx': min(i + self.chunk_size - 1, len(sentences) - 1)
            })
        logger.info(f"Создано {len(chunks)} чанков")
        return chunks

    async def _call_gemini_api(self, prompt: str, max_retries: int = 3) -> Optional[str]:
        """
        Вызов Gemini API с обработкой ошибок и fallback на локальную LLM
        
        Args:
            prompt: Промпт для модели
            max_retries: Максимальное количество попыток
            
        Returns:
            Ответ модели или None
        """
        if not self.gemini_available:
            return await self._call_local_llm(prompt)
        for attempt in range(max_retries):
            try:
                start_time = time.time()
                current_key = self.gemini_api_keys[self.current_key_index]
                self.key_usage_count[current_key] += 1
                self.key_last_used[current_key] = time.time()
                response = await asyncio.wait_for(
                    asyncio.to_thread(
                        self.gemini_model.generate_content,
                        prompt,
                        generation_config=self.generation_config
                    ),
                    timeout=self.fallback_timeout
                )
                if response and response.text:
                    logger.info(f"Gemini ответил за {time.time() - start_time:.2f}s")
                    return response.text.strip()
                else:
                    logger.warning("Gemini вернул пустой ответ")  
            except asyncio.TimeoutError:
                logger.warning(f"Gemini превысил таймаут {self.fallback_timeout}s, переключаемся на локальную LLM")
                return await self._call_local_llm(prompt)
            except Exception as e:
                error_str = str(e).lower()
                if any(keyword in error_str for keyword in ['quota', 'limit', 'rate']):
                    logger.warning(f"Лимит API ключа #{self.current_key_index + 1}, переключаемся...")
                    self._rotate_api_key()
                    if attempt == max_retries - 1:
                        logger.warning("Все API ключи исчерпаны, переключаемся на локальную LLM")
                        return await self._call_local_llm(prompt)
                else:
                    logger.error(f"Ошибка Gemini на попытке {attempt + 1}: {e}")
                    if attempt == max_retries - 1:
                        logger.warning("Все попытки Gemini исчерпаны, переключаемся на локальную LLM")
                        return await self._call_local_llm(prompt)
                await asyncio.sleep(2 ** attempt)
        return await self._call_local_llm(prompt)

    async def _call_local_llm(self, prompt: str) -> Optional[str]:
        """
        Вызов локальной LLM
        
        Args:
            prompt: Промпт для модели
            
        Returns:
            Ответ модели или None
        """
        if not self.local_llm:
            logger.error("Локальная LLM недоступна")
            return None
        try:
            logger.info("Используем локальную LLM")
            response = await asyncio.to_thread(
                self.local_llm.generate,
                prompt,
                max_tokens=1000,
                temp=0.1
            )
            return response.strip() if response else None
        except Exception as e:
            logger.error(f"Ошибка локальной LLM: {e}")
            return None

    def _parse_entity_response(self, response: str) -> List[Dict]:
        """
        Парсинг ответа с сущностями
        
        Args:
            response: Ответ модели
            
        Returns:
            Список сущностей
        """
        logger.debug(f"Парсинг ответа сущностей: {response[:200]}...")
        if not response or len(response.strip()) < 10:
            logger.warning("Получен слишком короткий ответ")
            return []
        entities = []
        if '---' in response:
            entity_blocks = response.split('---')
            for block in entity_blocks:
                entity = self._parse_entity_block(block.strip())
                if entity:
                    entities.append(entity)
        else:
            entities = self._parse_entities_line_by_line(response)
        logger.info(f"Распарсено {len(entities)} сущностей")
        if len(entities) == 0:
            logger.warning(f"Не удалось распарсить сущности из ответа: {response[:500]}")
        return entities
    
    def _parse_entity_block(self, block: str) -> Optional[Dict]:
        """
        Парсинг блока сущности
        
        Args:
            block: Блок текста с сущностью
            
        Returns:
            Словарь сущности или None
        """
        if not block:
            return None
        entity = {}
        lines = [line.strip() for line in block.split('\n') if line.strip()]
        for line in lines:
            if ':' not in line:
                continue
            key, value = line.split(':', 1)
            key = key.lower().strip()
            value = value.strip()
            if not value:
                continue
            if key in ['name', 'название', 'понятие']:
                entity['name'] = value
            elif key in ['type', 'тип']:
                entity['type'] = value
            elif key in ['description', 'описание']:
                entity['description'] = value
        if 'name' not in entity:
            return None
        if 'type' not in entity:
            entity['type'] = 'concepts'
        if 'description' not in entity:
            entity['description'] = ''
        return entity

    def _parse_entities_line_by_line(self, response: str) -> List[Dict]:
        """
        Построчный парсинг сущностей (fallback метод)
        
        Args:
            response: Ответ модели
            
        Returns:
            Список сущностей
        """
        entities = []
        lines = [line.strip() for line in response.split('\n') if line.strip()]
        current_entity = {}
        for line in lines:
            line = line.strip()
            if not line or line.startswith('#') or line.startswith('*'):
                continue
            if any(line.lower().startswith(prefix) for prefix in ['name:', 'название:', 'понятие:']):
                if current_entity and 'name' in current_entity:
                    entities.append(current_entity)
                name = line.split(':', 1)[1].strip() if ':' in line else line.strip()
                if name:
                    current_entity = {'name': name}
            elif any(line.lower().startswith(prefix) for prefix in ['type:', 'тип:']):
                if current_entity and ':' in line:
                    entity_type = line.split(':', 1)[1].strip()
                    if entity_type:
                        current_entity['type'] = entity_type
            elif any(line.lower().startswith(prefix) for prefix in ['description:', 'описание:']):
                if current_entity and ':' in line:
                    description = line.split(':', 1)[1].strip()
                    if description:
                        current_entity['description'] = description
        if current_entity and 'name' in current_entity:
            entities.append(current_entity)
        for entity in entities:
            if 'type' not in entity:
                entity['type'] = 'concepts'
            if 'description' not in entity:
                entity['description'] = ''
        return entities

    def _parse_relation_response(self, response: str) -> List[Dict]:
        """
        Улучшенный парсинг ответа с отношениями
        
        Args:
            response: Ответ модели
            
        Returns:
            Список отношений
        """
        logger.debug(f"Парсинг ответа отношений: {response[:200]}...")
        if not response or len(response.strip()) < 10:
            logger.warning("Получен слишком короткий ответ для отношений")
            return []
        relations = []
        if '---' in response:
            relation_blocks = response.split('---')
            for block in relation_blocks:
                relation = self._parse_relation_block(block.strip())
                if relation:
                    relations.append(relation)
        else:
            relations = self._parse_relations_line_by_line(response)
        logger.info(f"Распарсено {len(relations)} отношений")
        if len(relations) == 0:
            logger.warning(f"Не удалось распарсить отношения из ответа: {response[:500]}")
        return relations

    def _parse_relation_block(self, block: str) -> Optional[Dict]:
        """
        Парсинг блока отношения
        
        Args:
            block: Блок текста с отношением
            
        Returns:
            Словарь отношения или None
        """
        if not block:
            return None
        relation = {}
        lines = [line.strip() for line in block.split('\n') if line.strip()]
        for line in lines:
            if ':' not in line:
                continue
            key, value = line.split(':', 1)
            key = key.lower().strip()
            value = value.strip()
            if not value:
                continue
            if key in ['source', 'источник']:
                relation['source'] = value
            elif key in ['target', 'цель']:
                relation['target'] = value
            elif key in ['type', 'тип']:
                relation['type'] = value
            elif key in ['description', 'описание']:
                relation['description'] = value
        if 'source' not in relation or 'target' not in relation:
            return None
        if 'type' not in relation:
            relation['type'] = 'connected-to'
        if 'description' not in relation:
            relation['description'] = ''
        return relation

    def _parse_relations_line_by_line(self, response: str) -> List[Dict]:
        """
        Построчный парсинг отношений (fallback метод)
        
        Args:
            response: Ответ модели
            
        Returns:
            Список отношений
        """
        relations = []
        lines = [line.strip() for line in response.split('\n') if line.strip()]
        current_relation = {}
        for line in lines:
            line = line.strip()
            if not line or line.startswith('#') or line.startswith('*'):
                continue  
            if any(line.lower().startswith(prefix) for prefix in ['source:', 'источник:']):
                if current_relation and 'source' in current_relation:
                    relations.append(current_relation)
                source = line.split(':', 1)[1].strip() if ':' in line else line.strip()
                if source:
                    current_relation = {'source': source}
            elif any(line.lower().startswith(prefix) for prefix in ['target:', 'цель:']):
                if current_relation and ':' in line:
                    target = line.split(':', 1)[1].strip()
                    if target:
                        current_relation['target'] = target
            elif any(line.lower().startswith(prefix) for prefix in ['type:', 'тип:']):
                if current_relation and ':' in line:
                    relation_type = line.split(':', 1)[1].strip()
                    if relation_type:
                        current_relation['type'] = relation_type
            elif any(line.lower().startswith(prefix) for prefix in ['description:', 'описание:']):
                if current_relation and ':' in line:
                    description = line.split(':', 1)[1].strip()
                    if description:
                        current_relation['description'] = description
        if current_relation and 'source' in current_relation:
            relations.append(current_relation)
        for relation in relations:
            if 'type' not in relation:
                relation['type'] = 'connected-to'
            if 'description' not in relation:
                relation['description'] = ''
        return relations

    async def _extract_with_llm(self, chunk: Dict, prompt_template: str, entities_list: List[str] = None) -> List[Dict]:
        """
        Извлечение данных через LLM (Gemini или локальную)
        
        Args:
            chunk: Чанк для обработки
            prompt_template: Шаблон промпта
            entities_list: Список сущностей (для отношений)
            
        Returns:
            Список извлеченных данных
        """
        try:
            if entities_list:
                entities_str = ', '.join(entities_list)
                full_prompt = prompt_template.format(text=chunk['text'], entities=entities_str)
            else:
                full_prompt = prompt_template.format(text=chunk['text'])
            logger.info(f"Обрабатываю чанк {chunk['id']}: {chunk['text'][:50]}...")
            response = await self._call_gemini_api(full_prompt)
            if not response:
                logger.warning(f"Пустой ответ для чанка {chunk['id']}")
                return []
            if entities_list:
                parsed_data = self._parse_relation_response(response)
            else:
                parsed_data = self._parse_entity_response(response)
            if parsed_data:
                logger.info(f"Успешно извлечено {len(parsed_data)} элементов из чанка {chunk['id']}")
                return parsed_data
            else:
                logger.warning(f"Не удалось извлечь данные из чанка {chunk['id']}")
                logger.debug(f"Ответ от LLM: {response}")
                return []
                
        except Exception as e:
            logger.error(f"Ошибка обработки чанка {chunk['id']}: {str(e)}")
            logger.debug(f"Подробности ошибки:", exc_info=True)
            return []

    # async def _extract_entities_from_chunks(self, chunks: List[Dict]) -> List[Entity]:
    #     """
    #     Извлечение сущностей из чанков
        
    #     Args:
    #         chunks: Список чанков
            
    #     Returns:
    #         Список сущностей
    #     """
    #     logger.info("=== ЭТАП 1: Извлечение сущностей ===")
    #     all_entities = []
        
    #     for chunk in chunks:
    #         logger.info(f"Обрабатываю чанк {chunk['id'] + 1}/{len(chunks)}")
    #         try:
    #             entities_data = await self._extract_with_llm(chunk, self.entity_prompt)
    #             chunk_entities = []
    #             for i, entity_data in enumerate(entities_data):
    #                 if 'name' in entity_data and entity_data['name']:
    #                     entity = Entity(
    #                         id=f"e_{chunk['id']}_{i}",
    #                         name=entity_data['name'].strip(),
    #                         type=entity_data.get('type', 'concepts').strip(),
    #                         description=entity_data.get('description', '').strip(),
    #                         source_chunks=[chunk['id']]
    #                     )
    #                     chunk_entities.append(entity)
    #             all_entities.extend(chunk_entities)
    #             logger.info(f"Найдено сущностей в чанке: {len(chunk_entities)}")
    #             await asyncio.sleep(1)
    #         except Exception as e:
    #             logger.error(f"Критическая ошибка при обработке энтити в чанке {chunk['id']}: {str(e)}")
    #             continue
    #     unique_entities = self._merge_duplicate_entities(all_entities)
    #     logger.info(f"Общее количество уникальных энтити: {len(unique_entities)}")
    #     return unique_entities

    # async def _extract_relations_from_chunks(self, chunks: List[Dict], entities: List[Entity]) -> List[Relation]:
    #     """
    #     Извлечение отношений из чанков
        
    #     Args:
    #         chunks: Список чанков
    #         entities: Список сущностей
            
    #     Returns:
    #         Список отношений
    #     """
    #     logger.info("=== ЭТАП 2: Извлечение отношений ===")
    #     entity_names = [entity.name for entity in entities]
    #     entity_name_to_obj = {entity.name: entity for entity in entities}
    #     all_relations = []
    #     for chunk in chunks:
    #         logger.info(f"Обрабатываю отношения в чанке {chunk['id'] + 1}/{len(chunks)}")
    #         try:
    #             relations_data = await self._extract_with_llm(chunk, self.relation_prompt, entity_names)
    #             chunk_relations = []
                    
    #             for i, relation_data in enumerate(relations_data):
    #                 if ('source' in relation_data and 'target' in relation_data and 
    #                     relation_data['source'] and relation_data['target']):
                            
    #                     source_name = relation_data['source'].strip()
    #                     target_name = relation_data['target'].strip()
                            
    #                     if source_name in entity_name_to_obj and target_name in entity_name_to_obj:
    #                         relation = Relation(
    #                                 source=source_name,
    #                                 target=target_name,
    #                                 type=relation_data.get('type', 'connected-to').strip(),
    #                                 description=relation_data.get('description', '').strip(),
    #                                 source_chunks=[chunk['id']]
    #                             )
    #                         chunk_relations.append(relation)
    #                     else:
    #                         logger.debug(f"Пропускаю отношение {source_name} -> {target_name}: сущности не найдены")
                    
    #             all_relations.extend(chunk_relations)
    #             logger.info(f"Найдено отношений в чанке: {len(chunk_relations)}")
    #             await asyncio.sleep(1)
    #         except Exception as e:
    #             logger.error(f"Критическая ошибка при обработке отношения в чанке {chunk['id']}: {str(e)}")
    #             continue    
    #     unique_relations = self._merge_duplicate_relations(all_relations)
    #     logger.info(f"Общее количество уникальных отношении: {len(unique_relations)}")
    #     return unique_relations 

    async def _extract_entities_from_chunks(self, chunks: List[Dict]) -> List[Entity]:
        logger.info("=== ЭТАП 1: Извлечение сущностей ===")
        if not chunks:
            logger.warning("Пустой список чанков")
            return []
        all_entities = []
        chunks_count = len(chunks)
        for i, chunk in enumerate(chunks):
            chunk_id = chunk['id']
            logger.info(f"Обрабатываю чанк {chunk_id + 1}/{chunks_count}")
            try:
                entities_data = await self._extract_with_llm(chunk, self.entity_prompt)
                chunk_entities = [
                    Entity(
                        id=f"e_{chunk_id}_{j}",
                        name=entity_data['name'].strip(),
                        type=entity_data.get('type', 'concepts').strip(),
                        description=entity_data.get('description', '').strip(),
                        source_chunks=[chunk_id]
                    )
                    for j, entity_data in enumerate(entities_data)
                    if entity_data.get('name') and entity_data['name'].strip()
                ]
                all_entities.extend(chunk_entities)
                logger.info(f"Найдено сущностей в чанке: {len(chunk_entities)}")
                if (i + 1) % 5 == 0 or i == chunks_count - 1:
                    await asyncio.sleep(1)      
            except Exception as e:
                logger.error(f"Критическая ошибка при обработке энтити в чанке {chunk_id}: {str(e)}")
                continue
        unique_entities = self._merge_duplicate_entities(all_entities)
        logger.info(f"Общее количество уникальных энтити: {len(unique_entities)}")
        return unique_entities

    async def _extract_relations_from_chunks(self, chunks: List[Dict], entities: List[Entity]) -> List[Relation]:
        logger.info("=== ЭТАП 2: Извлечение отношений ===")
        if not chunks or not entities:
            logger.warning("Пустой список чанков или сущностей")
            return []
        entity_names_set = {entity.name for entity in entities}
        all_relations = []
        chunks_count = len(chunks)
        for i, chunk in enumerate(chunks):
            chunk_id = chunk['id']
            logger.info(f"Обрабатываю отношения в чанке {chunk_id + 1}/{chunks_count}")
            try:
                relations_data = await self._extract_with_llm(
                    chunk, self.relation_prompt, list(entity_names_set)
                )
                chunk_relations = []
                for j, relation_data in enumerate(relations_data):
                    if not (relation_data.get('source') and relation_data.get('target')):
                        continue
                    source_name = relation_data['source'].strip()
                    target_name = relation_data['target'].strip()
                    if source_name in entity_names_set and target_name in entity_names_set:
                        relation = Relation(
                            source=source_name,
                            target=target_name,
                            type=relation_data.get('type', 'connected-to').strip(),
                            description=relation_data.get('description', '').strip(),
                            source_chunks=[chunk_id]
                        )
                        chunk_relations.append(relation)
                    else:
                        logger.debug(f"Пропускаю отношение {source_name} -> {target_name}: сущности не найдены")
                all_relations.extend(chunk_relations)
                logger.info(f"Найдено отношений в чанке: {len(chunk_relations)}")
                if (i + 1) % 5 == 0 or i == chunks_count - 1:
                    await asyncio.sleep(1) 
            except Exception as e:
                logger.error(f"Критическая ошибка при обработке отношения в чанке {chunk_id}: {str(e)}")
                continue
        unique_relations = self._merge_duplicate_relations(all_relations)
        logger.info(f"Общее количество уникальных отношений: {len(unique_relations)}")
        return unique_relations
    
    def _merge_duplicate_entities(self, entities: List[Entity]) -> List[Entity]:
        """
        Объединяет дублирующиеся сущности
        
        Args:
            entities: Список сущностей
            
        Returns:
            Список уникальных сущностей
        """
        entity_groups = defaultdict(list)
        for entity in entities:
            key = self._normalize_entity_key(entity.name)
            entity_groups[key].append(entity)
        merged_entities = []
        for name_key, group in entity_groups.items():
            if len(group) == 1:
                merged_entities.append(group[0])
            else:
                base_entity = group[0]
                all_chunks = set()
                descriptions = []
                for entity in group:
                    all_chunks.update(entity.source_chunks)
                    if entity.description and entity.description not in descriptions:
                        descriptions.append(entity.description)
                base_entity.source_chunks = sorted(list(all_chunks))
                base_entity.description = '; '.join(descriptions) if descriptions else base_entity.description
                merged_entities.append(base_entity)
        return merged_entities

    def _merge_duplicate_relations(self, relations: List[Relation]) -> List[Relation]:
        """
        Объединяет дублирующиеся отношения
        
        Args:
            relations: Список отношений
            
        Returns:
            Список уникальных отношений
        """
        relation_groups = defaultdict(list)
        for relation in relations:
            key = f"{self._normalize_entity_key(relation.source)}|{self._normalize_entity_key(relation.target)}|{relation.type.lower()}"
            relation_groups[key].append(relation)
        merged_relations = []
        for relation_key, group in relation_groups.items():
            if len(group) == 1:
                merged_relations.append(group[0])
            else:
                base_relation = group[0]
                all_chunks = set()
                descriptions = []
                for relation in group:
                    all_chunks.update(relation.source_chunks)
                    if relation.description and relation.description not in descriptions:
                        descriptions.append(relation.description)
                base_relation.source_chunks = sorted(list(all_chunks))
                base_relation.description = '; '.join(descriptions) if descriptions else base_relation.description
                merged_relations.append(base_relation)
        return merged_relations

    def _normalize_entity_key(self, entity_name: str) -> str:
        """
        Нормализация названия сущности для группировки
        
        Args:
            entity_name: Название сущности
            
        Returns:
            Нормализованное название
        """
        normalized = re.sub(r'\s+', ' ', entity_name.lower().strip())
        normalized = re.sub(r'[^\w\s]', '', normalized)
        
        return normalized

    async def extract_knowledge_graph(self, text: str) -> Dict[str, Any]:
        """
        Основной метод для создания графа знаний из текста
        
        Args:
            text: Входной текст для анализа
            
        Returns:
            Словарь с графом знаний
        """
        logger.info("=== НАЧАЛО ОБРАБОТКИ ТЕКСТА ===")
        sentences = self.split_text_into_sentences(text)
        logger.info(f"Текст разделен на {len(sentences)} предложений")
        if len(sentences) > self.max_sentences:
            logger.info(f"Текст слишком большой ({len(sentences)} предложений). Обрезаем до {self.max_sentences}.")
            sentences = sentences[:self.max_sentences]
        chunks = self._create_chunks(sentences)
        logger.info(f"Создано чанков: {len(chunks)} (по {self.chunk_size} предложений)")
        entities = await self._extract_entities_from_chunks(chunks)
        relations = []
        if entities:
            relations = await self._extract_relations_from_chunks(chunks, entities)
        else:
            logger.warning("Сущности не найдены, пропускаем поиск отношений")
        entity_types = defaultdict(int)
        for entity in entities:
            entity_types[entity.type] += 1
        relation_types = defaultdict(int)
        for relation in relations:
            relation_types[relation.type] += 1
        result = {
            'entities': [asdict(e) for e in entities],
            'relations': [asdict(r) for r in relations],
            'chunks_info': [
                {
                    'id': chunk['id'],
                    'text_preview': chunk['text'][:100] + '...' if len(chunk['text']) > 100 else chunk['text'],
                    'sentences_count': len(chunk['sentences']),
                    'has_formulas': bool(self.formula_pattern.search(chunk['original_text']))
                } for chunk in chunks
            ],
            'stats': {
                'total_sentences': len(sentences),
                'total_chunks': len(chunks),
                'chunk_size': self.chunk_size,
                'total_entities': len(entities),
                'total_relations': len(relations),
                'entity_types': dict(entity_types),
                'relation_types': dict(relation_types),
                'gemini_usage': self.key_usage_count,
                'local_llm_used': not self.gemini_available
            }
        }
        logger.info("=== ОБРАБОТКА ЗАВЕРШЕНА ===")
        logger.info(f"Найдено сущностей: {len(entities)}")
        logger.info(f"Найдено отношений: {len(relations)}")
        logger.info(f"Использование Gemini: {self.key_usage_count}")
        return result

In [37]:

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class SynonymGroup:
    """Класс для представления группы синонимов"""
    main_term: str
    synonyms: List[str]
    confidence: float
    source: str 

class SynonymExtractor:
    """Класс для извлечения синонимов из энтити с помощью LLM"""
    def __init__(self, graphembedder: ImprovedKnowledgeExtractor,
                 max_concurrent: int = 5,
                 ):
        """
        Инициализация экстрактора синонимов
        Args:
            graphembedder: Экземпляр ImprovedKnowledgeExtractor
            max_concurrent: Максимальное количество одновременных запросов
        """
        self.graphembedder = graphembedder
        self.max_concurrent = max_concurrent
        self.cache = {}
    
    def _create_synonym_prompt(self, entities: List[Dict], context: str = "") -> str:
        """Создание промпта для извлечения синонимов"""

        base_prompt = """Ты - эксперт по терминологии в области машинного обучения, нейронных сетей и когнитивных наук.

Задача: Для каждого предоставленного термина найди все возможные синонимы, переводы и альтернативные названия.

Контекст исследования: {context}

Термины для анализа:
{entity_list}

Требования к ответу:
1. Для каждого термина укажи ВСЕ возможные синонимы и однокоренные слова
2. Включи переводы на английский/русский
3. Включи сокращения и аббревиатуры
4. Включи научные и разговорные варианты
5. Укажи уровень уверенности (1-10)

Формат ответа (строго JSON):
{{
  "synonyms": [
    {{
      "main_term": "основной термин",
      "alternatives": ["синоним1", "синоним2", "перевод", "сокращение"],
      "confidence": 8
    }}
  ]
}}

ВАЖНО! main_term должен быть всегда уникален по смыслу и написанию, то что в alternatives должно иметь разное написание но похожее по смыслу!
Примеры качественных синонимов:
- "нейронная сеть" → ["neural network", "нейросеть", "ИНС", "искусственная нейронная сеть"]
- "функция активации" → ["activation function", "функция активации", "активация"]
- "машинное обучение" → ["machine learning", "ML", "автоматическое обучение"]"""
        
        entity_descriptions = []
        for i, entity in enumerate(entities, 1):
            desc = f"{i}. '{entity['name']}' ({entity.get('type', 'unknown')})"
            if entity.get('description'):
                desc += f" - {entity['description'][:100]}..."
            entity_descriptions.append(desc)
        entity_list = "\n".join(entity_descriptions)
        
        return base_prompt.format(
            context=context or "исследование механизмов мозга и нейронных сетей",
            entity_list=entity_list
        )
    
    async def _make_llm_request(self, prompt: str) -> Optional[Dict]:
        """Выполнение запроса к LLM"""
        try:
            if asyncio.iscoroutinefunction(self.graphembedder._call_gemini_api):
                return await self.graphembedder._call_gemini_api(prompt)
            else:
                return self.graphembedder._call_gemini_api(prompt)
        except Exception as e:
            logger.error(f"Ошибка LLM запроса: {e}")
            return None
    
    def _batch_entities(self, entities: List[Dict], batch_size: int = 10) -> List[List[Dict]]:
        """Разбиение энтити на батчи для обработки"""
        batches = []
        for i in range(0, len(entities), batch_size):
            batches.append(entities[i:i + batch_size])
        return batches
    
    async def extract_synonyms_batch(self, 
                                   entities: List[Dict], 
                                   context: str = "",
                                   batch_size: int = 10) -> List[SynonymGroup]:
        """
        Извлечение синонимов для батча энтити
        
        Args:
            entities: Список энтити
            context: Контекст для лучшего понимания
            batch_size: Размер батча для обработки
            
        Returns:
            Список групп синонимов
        """
        logger.info(f"Начинаю извлечение синонимов для {len(entities)} энтити")
        batches = self._batch_entities(entities, batch_size)
        synonym_groups = []
        semaphore = asyncio.Semaphore(self.max_concurrent)
        
        async def process_batch(batch: List[Dict]) -> List[SynonymGroup]:
            async with semaphore:
                try:
                    prompt = self._create_synonym_prompt(batch, context)

                    response = await self._make_llm_request(prompt)
                    
                    if not response or not isinstance(response, dict) or 'synonyms' not in response:
                        logger.warning(f"Некорректный ответ от LLM: {response}")
                        return []
                    
                    groups = []
                    for syn_data in response['synonyms']:
                        if 'main_term' in syn_data and 'alternatives' in syn_data:
                            group = SynonymGroup(
                                main_term=syn_data['main_term'],
                                synonyms=syn_data['alternatives'],
                                confidence=syn_data.get('confidence', 5) / 10.0,
                                source='llm'
                            )
                            groups.append(group)
                    
                    return groups
                    
                except Exception as e:
                    logger.error(f"Ошибка в process_batch: {e}")
                    return []

        tasks = [process_batch(batch) for batch in batches]

        batch_results = await asyncio.gather(*tasks, return_exceptions=True)

        for result in batch_results: 
            if isinstance(result, Exception):
                logger.error(f"Ошибка обработки батча: {result}")
                continue
            synonym_groups.extend(result)
        
        logger.info(f"Извлечено {len(synonym_groups)} групп синонимов")
        return synonym_groups
    
    # def create_synonym_mapping(self, synonym_groups: List[SynonymGroup], 
    #                          confidence_threshold: float = 0.6) -> Dict[str, List[str]]:
    #     """
    #     Создание маппинга синонимов для использования в дедупликации
        
    #     Args:
    #         synonym_groups: Группы синонимов
    #         confidence_threshold: Минимальный уровень уверенности
            
    #     Returns:
    #         Словарь синонимов
    #     """
    #     synonym_mapping = {}
        
    #     for group in synonym_groups:
    #         if group.confidence >= confidence_threshold:
    #             main_term = group.main_term.lower().strip()
    #             synonyms = [syn.lower().strip() for syn in group.synonyms if syn.strip()] 
                
    #             if main_term and synonyms:
    #                 synonym_mapping[main_term] = synonyms

    #                 for synonym in synonyms:
    #                     if synonym not in synonym_mapping:
    #                         synonym_mapping[synonym] = [main_term]
    #                     elif main_term not in synonym_mapping[synonym]:
    #                         synonym_mapping[synonym].append(main_term)
        
    #     return synonym_mapping


    def create_synonym_mapping(self, synonym_groups: List[SynonymGroup],
                                    confidence_threshold: float = 0.6) -> Dict[str, List[str]]:
        if not synonym_groups:
            return {}
        from collections import defaultdict
        synonym_sets = defaultdict(set)
        for group in synonym_groups:
            if group.confidence < confidence_threshold:
                continue  
            main_term = group.main_term.lower().strip()
            if not main_term:
                continue
            valid_synonyms = {
                syn.lower().strip() 
                for syn in group.synonyms 
                if syn and syn.strip()
            }
            if not valid_synonyms:
                continue
            synonym_sets[main_term].update(valid_synonyms)
            for synonym in valid_synonyms:
                synonym_sets[synonym].add(main_term)
        return {
            term: list(synonyms) 
            for term, synonyms in synonym_sets.items() 
            if synonyms
        }

async def extract_synonyms_for_entities(graphembedder: ImprovedKnowledgeExtractor, 
                                      entities: List[Dict], 
                                      context: str = "") -> Dict[str, List[str]]:
    """
    Основная функция для извлечения синонимов
    
    Args:
        graphembedder: Экземпляр ImprovedKnowledgeExtractor
        entities: Список энтити
        context: Контекст исследования
        
    Returns:
        Словарь синонимов для использования в дедупликации
    """
    extractor = SynonymExtractor(
        graphembedder=graphembedder, 
        max_concurrent=3
    )
    
    synonym_groups = await extractor.extract_synonyms_batch(
        entities=entities,
        context=context,
        batch_size=8  
    )
    
    synonym_mapping = extractor.create_synonym_mapping(
        synonym_groups, 
        confidence_threshold=0.6
    )
    
    return synonym_mapping

In [38]:
@dataclass
class EntityMatch:
    """Класс для представления совпадения энтити"""
    entity1_id: str
    entity2_id: str
    similarity_score: float
    match_type: str  

class EntityDeduplicator:
    def __init__(self, graphembedder: ImprovedKnowledgeExtractor, use_llm=False):
        self.use_llm = use_llm
        self.graphembedder = graphembedder
        try:
            self.nlp = spacy.load("ru_core_news_sm")
        except OSError:
            print("Модель ru_core_news_sm не найдена. Установите: python -m spacy download ru_core_news_sm")
            self.nlp = None
         
        if use_llm:
            try:
                self.sentence_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
            except:
                print("Не удалось загрузить модель sentence-transformers")
                self.sentence_model = None

        self.synonyms = {
            'функция': ["функцией","функции",'function','функц',"mapping","отображение","зависимость","соотношение","преобразование","transformation","relation","mathematical function","математическая функция","ф-ция"],
            'предсказание': ["предсказании","предсказнием",'prediction', 'прогноз', 'предикт',"inference","вывод","прогноз","оценка","estimation","прогнозирование","вычисление","computation","предсказание значения","предсказание результата","результат работы сети","выход","output"],
            'слой': ["слое","слоем",'layer', 'уровень',"ярус","level","пласт","stratum","слой нейронов","neural layer","hidden layer","скрытый слой","input layer","входной слой","output layer","выходной слой"],
            'нейрон': ["нейроне","нейроном",'neuron', 'нервная клетка',"neural cell","нервная клетка","artificial neuron","искусственный нейрон"],
            'вес': ["весом","весе",'weight',"connection weight","synaptic weight","синаптический вес","весовой коэффициент","weight coefficient","параметр связи","connection parameter","сила связи","connection strength","вес","weighting factor"],
            'смещение': ["смещении","смещением",'bias', 'сдвиг',"bias term","offset","bias unit","смещающий фактор","bias factor"],
            'активация': ['activation',"activation function","transfer function","передаточная функция","nonlinear function","нелинейная функция активации","activation function (af)","функция активации (фа)","activation unit","блок активации"],
            'матрица': ['matrix',"матрицей", "матрице"],
            'вектор': ['vector',"всетором","векторе"],
            'оптимизация': ['optimization', 'оптимизац', "оптимизации"],
            "метод оптимизации параметров": [
                "parameter optimization method",
                "optimization algorithm",
                "parameter tuning",
                "parameter estimation",
                "оптимизация параметров",
                "настройка параметров",
                "оценка параметров",
                "алгоритм оптимизации",
                "метод настройки параметров",
                "parameter optimization technique"
            ],
            'обучение': ['training',
                          'learning',
                            'машинное обучение',
                              'machine learning',
                                "обучении"],
            "функция ошибки": [
                "loss function",
                "cost function",
                "objective function",
                "error function",
                "функция потерь",
                "целевая функция",
                "функционал ошибки",
                "критерий ошибки",
                "мера ошибки",
                "функция стоимости",
                "функция расхождения",
                "error",
                "loss",
                "cost",
                "objective",
                "потеря",
                "ошибка",
                "стоимость",
                "объектив",
                "функция несовпадения",
                "функция неточности",
                "функция несоответствия"
            ],
            "средняя функция ошибки": [
                "mean squared error (mse)",
                "mean error",
                "average loss",
                "average cost",
                "mse",
                "среднеквадратичная ошибка",
                "средняя ошибка",
                "усредненная функция потерь",
                "усредненная функция стоимости",
                "среднее значение функции ошибки",
                "average error function",
                "mean loss function",
                "mean cost function"
            ],          
            "градиентный спуск": [
                "gradient descent",
                "gd",
                "steepest descent",
                "метод градиентного спуска",
                "спуск по градиенту",
                "градиентный метод",
                "метод наискорейшего спуска",
                "gradient descent optimization",
                "gradient optimization",
                "градиентный спуск",
                "алгоритм градиентного спуска",
                "оптимизация градиентом",
                "метод оптимизации первого порядка",
                "gradient method"
            ],
            "алгоритмы": [
                "algorithms",
                "methods",
                "approaches",
                "techniques",
                "procedures",
                "методы",
                "подходы",
                "техники",
                "процедуры",
                "способы",
                "рецепты",
                "инструкции",
                "набор инструкций",
                "набор методов"
            ],
            "метод обратной связи": [
                "feedback method",
                "feedback control",
                "closed-loop control",
                "обратная связь",
                "управление с обратной связью",
                "регулирование с обратной связью",
                "замкнутый контур управления",
                "feedback mechanism",
                "обратная связь",
                "обратное распространение"
            ],
            "коэффициент обучения": [
                "learning rate",
                "step size",
                "coefficient of learning",
                "скорость обучения",
                "размер шага",
                "learning coefficient",
                "step size parameter"
            ],
            "ib-искажение": [
                "ib distortion",
                "искажение информационного бутылочного горлышка",
                "distortion of the information bottleneck",
                "мера искажения в ib",
                "потеря информации в представлении",
                "информационные потери",
                "information loss",
                "information bottleneck distortion",
                "ib-loss",
                "мера релевантности представления"
            ],
            "ibm": [
                "information bottleneck method",
                "метод информационного бутылочного горлышка",
                "информационный барьер",
                "information bottleneck",
                "ib",
                "метод ib",
                "подход ib",
                "алгоритм ib",
                "information bottleneck principle"
            ], 
            "классификация": [
                "категоризация",
                "распознавание образов",
                "отнесение к классу",
                "кластеризация (в контексте обучения с учителем)",
                "classification",
                "categorization",
                "pattern recognition",
                "clustering (supervised learning context)",
                "классификация",
                "категоризация",
                "распознавание образов",
                "кластеризация"
            ],
            "регрессия": [
                "регрессия",
                "регрессионный анализ",
                "восстановление зависимости",
                "regression",
                "regression analysis",
                "function approximation",
                "регрессия",
                "регрессионный анализ",
                "аппроксимация функции"
            ],
            "линейная регрессия": [
                "linear regression",
                "lr",
                "линейная модель",
                "linear model",
                "регрессия по методу наименьших квадратов",
                "least squares regression",
                "обыкновенная линейная регрессия",
                "ordinary linear regression",
                "olr"
            ],
            "knn": [
                "k-ближайших соседей",
                "метод k-ближайших соседей",
                "k-nn",
                "k-nearest neighbors",
                "knn",
                "k-ближайших соседей",
                "метод ближайших соседей"
            ],
            "качество данных": [
                "quality of data",
                "точность данных",
                "согласованность данных",
                "чистота данных",
                "data quality",
                "data integrity",
                "data accuracy",
                "data consistency",
                "data completeness",
                "качество данных",
                "целостность данных",
                "точность данных",
                "согласованность данных",
                "полнота данных"
            ],
            "обобщающая способность": [
                "generalization ability",
                "способность к обобщению",
                "способность к генерализации",
                "устойчивость модели",
                "generalization",
                "generalizability",
                "out-of-sample performance",
                "мощность модели",
                "обобщение",
                "генерализация",
                "устойчивость",
                "способность к обучению"
            ],
            "информативные признаки": [
                "informative patterns",
                "полезные закономерности",
                "значимые зависимости",
                "сигналы",
                "informative features",
                "useful patterns",
                "meaningful relationships",
                "signals",
                "информативные закономерности",
                "полезные признаки",
                "значимые закономерности",
                "сигналы"
            ],
            "зашумленные данные" : [ 
                "шумовые данные",
                "нерелевантные данные",
                "лишние данные",
                "мусорные данные",
                "noise",
                "noise data",
                "noisy data",
                "irrelevant data",
                "garbage data",
                "шум",
                "нерелевантные данные",
                "мусор"
            ],
            "k-means": [
                "k-means algorithm",
                "k-средних",
                "k-means clustering",
                "k-средних кластеризация",
                "метод k-средних",
                "k-means",
                "алгоритм кластеризации k-средних"
            ],
            "кластеризация данных": [
                "data clustering",
                "clustering",
                "кластерный анализ",
                "группировка данных",
                "сегментация данных",
                "data segmentation",
                "data grouping",
                "cluster analysis"
            ],
            "кластеры": [
                "k clusters",
                "кластеры",
                "группы",
                "k groups",
                "сегменты",
                "clusters",
                "groups",
                "segments",
                "классы"
            ],
            "метод опорных векторов": [
                "support vector machine",
                "svm",
                "support vector networks",
                "метод опорных сетей",
                "машина опорных векторов",
                "support vector classifier",
                "svc",
                "supportvectormachines (svm)",
                "support vector machine",
                "метод опорных векторов",
                "машина опорных векторов",
                "support vector networks",
                "support vector classifier",
                "svc",
                "supportvectormachines (svm)",
                "сети опорных векторов",
                "support vector networks"
            ],
             "алгоритм случайного леса": [
                "random forest algorithm",
                "random forest",
                "случайный лес",
                "rf",
                "алгоритм rf",
                "random decision forest",
                "ансамбль деревьев решений"
            ],
            "входные данные": [
                "входные данные",
                "входной вектор",
                "признак",
                "фактор",
                "переменная",
                "независимая переменная",
                "предиктор",
                "input data",
                "input vector",
                "feature",
                "predictor",
                "independent variable",
                "explanatory variable",
                "input",
                "вход",
                "аргумент",
                "аргумент функции"
            ],
            "теория вероятностей": [
                "probability theory",
                "probability",
                "теория вероятности",
                "тв",
                "вероятностная теория",
                "исчисление вероятностей",
                "probability calculus",
                "probability science"
            ],
            "событие": [
                "event",
                "случайное событие",
                "исход",
                "наступление события",
                "реализация события"
                ],
            "классы": [
                "категории",
                "типы",
                "разряды",
                "группы",
                "множества",
                "классификации",
                "entities",
                "объекты",
                "categories",
                "types",
                "groups",
                "sets",
                "classifications",
                "классы объектов",
                "классы данных",
                "разделы",
                "разновидности"
            ],
            "условная энтропия x при условии t_ε" : [
                "$h(x|t_ε)$",
                "conditional entropy of x given t_ε",
                "энтропия условной вероятности",
                "мера неопределенности x при известном t_ε",
                "conditional entropy",
                "h(x|t_epsilon)",
                "h(x|tε)",
                "условная информационная энтропия",
                "information entropy"
            ],
            "взаимная информация между t_ε и x": [
                "информация о взаимодействии между t_ε и x",
                "mutual information between t_ε and x",
                "information interaction",
                "i(t_epsilon; x)",
                "i(tε; x)",
                "$i(t_ε; x)$",
                "передаваемая информация",
                "количество информации",
                "information gain"
            ],
            "cnn": [
                "сверточная нейронная сеть",
                "свёрточная нейросеть",
                "кнс",
                "convolutional net",
                "convnet",
                "снс",
                "deep convolutional neural networks",
                "dcnn"
            ],
            "края": [
                "границы",
                "линии",
                "контуры",
                "edges",
                "borders",
                "lines",
                "contours",
                "примитивы",
                "простые признаки",
                "характерные точки",
                "edge features",
                "edge detection"
            ],
            "долговременная память": [
                "long-term memory",
                "ltm",
                "веса нейронной сети",
                "параметры модели",
                "сохраненные знания",
                "матрицы весов",
                "вектора смещения",
                "bias",
                "weights",
                "model parameters",
                "trained parameters",
                "обученные параметры"
            ],       
            "информационная плоскость": [
                "information plane",
                "ib plane",
                "плоскость информационной бутылки",
                "information bottleneck plane",
                "область представления информации",
                "пространство информации",
                "information space",
                "information representation space",
                "плоскость обучения",
                "learning plane"
            ],
            "измерение полезной информации": [
                "ось полезной информации",
                "axis of relevant information",
                "relevant information dimension",
                "ось r",
                "r-axis",
                "ось представления информации",
                "information representation axis"
            ],
            "неопределенность": [
                "неопределённость",
                "неизвестность",
                "сомнение",
                "двусмысленность",
                "vagueness",
                "ambiguity",
                "indeterminacy",
                "lack of certainty",
                "degree of doubt",
                "uncertainty quantification (uq)",
                "энтропия (в контексте теории информации)",
                "риск (в контексте принятия решений)",
                "вероятностная неопределенность",
                "эпистемическая неопределенность",
                "алеаторная неопределенность"
            ],
            "слои нейронной сети": [
                "neural network layers",
                "layers of a neural network",
                "слои инс",
                "слои нейросети",
                "уровни нейронной сети",
            ],
            "скрытые слои": [
                "hidden layers",
                "скрытый слой",
                "hidden layer",
                "промежуточные слои",
                "intermediate layers",
                "внутренние слои",
                "internal layers",
                "слои между входом и выходом",
                "layers between input and output",
                "невидимые слои",
                "invisible layers"
            ],
            "разделимость данных": [
                "data separability",
                "separability of data",
                "линейная разделимость",
                "linearly separable data",
                "разделимость классов",
                "кластеризуемость",
                "отделимость данных",
                "различимость данных",
                "разделимость выборок",
                "разделимость признаков",
                "data distinguishability",
                "class separability",
                "feature separability",
                "linearly separable data",
                "линейно разделимые данные",
                "разделимые данные",
                "separable data",
                "данные с линейной разделимостью",
                "data with linear separability",
                "данные, которые можно разделить прямой линией",
                "data that can be separated by a straight line",
                "линейно разделимый набор данных",
                "linearly separable dataset"
            ],
            "наблюдатель": [
                "observer",
                "agent",
                "агент",
                "субъект",
                "пользователь",
                "система",
                "процессор",
                "датчик",
                "сенсор",
                "рецептор",
                "приёмник информации",
                "получатель информации",
                "информационный агент"
            ],
            "мера информации": [
                "information measure",
                "measure of information",
                "количество информации",
                "информационная энтропия",
                "информационная ценность",
                "информативность",
                "информационный критерий",
                "информационная метрика"
            ],
            "собственная информация": [
                "information content",
                "self-information",
                "собственная информация",
                "информационное содержание",
                "информативность события",
                "information of event x",
                "i(x)",
                "мера неожиданности события",
                "surprisal",
                "-log(p(x))",
                "отрицательный логарифм вероятности события"
            ],
            "априорная вероятностная модель": [
                "prior probability model",
                "модель априорных вероятностей",
                "apriori probability model",
                "начальная вероятностная модель",
                "initial probability model",
                "базовая вероятностная модель",
                "baseline probability model",
                "модель предварительной вероятности",
                "preliminary probability model",
                "априорная модель",
                "prior model"
            ],
            "выходные данные": [
                "целевой вывод $y$",
                "target output $y$",
                "output data $y$",
                "предсказанный язык",
                "predicted language",
                "определенный язык",
                "identified language",
                "метка класса",
                "class label",
                "истинный класс",
                "true class",
                "y"
            ],
            "извлечение признаков": [
                "feature extraction",
                "выделение признаков",
                "feature selection",
                "инженерия признаков",
                "feature engineering",
                "отбор признаков",
                "feature subset selection",
                "преобразование признаков",
                "feature transformation",
                "проектирование признаков",
                "feature design"
            ],
            "дерево решений": [
                "decision tree",
                "деревья решений",
                "решающее дерево",
                "решающие деревья",
                "decision tree learning",
                "обучение с использованием деревьев решений",
                "дерево классификации",
                "classification tree",
                "дерево регрессии",
                "regression tree",
                "дерево принятия решений",
                "decision making tree"
            ]
        }

    def normalize_text(self, text: str) -> str:
        """Нормализация текста"""
        if not text:
            return ""
        text = text.lower().strip()
        text = re.sub(r'[^\w\s-]', '', text)
        text = re.sub(r'\s+', ' ', text)
        return text
    
    def get_lemma(self, text: str) -> str:
        """Получение леммы текста"""
        if not self.nlp or not text:
            return text
        doc = self.nlp(text)
        lemmas = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
        return ' '.join(lemmas) if lemmas else text
    
    # def find_exact_matches(self, entities: List[Dict]) -> List[EntityMatch]:
    #     """Поиск точных совпадений по названию"""
    #     matches = []
    #     name_to_entities = defaultdict(list)
    #     for entity in entities:
    #         normalized_name = self.normalize_text(entity['name'])
    #         name_to_entities[normalized_name].append(entity['id'])
    #     for name, entity_ids in name_to_entities.items():
    #         if len(entity_ids) > 1:
    #             for i in range(len(entity_ids)):
    #                 for j in range(i + 1, len(entity_ids)):
    #                     matches.append(EntityMatch(
    #                         entity_ids[i], entity_ids[j], 1.0, 'exact'
    #                     ))
    #     return matches
    
    # def find_lemma_matches(self, entities: List[Dict]) -> List[EntityMatch]:
    #     """Поиск совпадений по леммам"""
    #     if not self.nlp:
    #         return []
    #     matches = []
    #     lemma_to_entities = defaultdict(list)
    #     for entity in entities:
    #         lemma = self.get_lemma(self.normalize_text(entity['name']))
    #         if lemma:
    #             lemma_to_entities[lemma].append(entity['id'])
    #     for lemma, entity_ids in lemma_to_entities.items():
    #         if len(entity_ids) > 1:
    #             for i in range(len(entity_ids)):
    #                 for j in range(i + 1, len(entity_ids)):
    #                     matches.append(EntityMatch(
    #                         entity_ids[i], entity_ids[j], 0.9, 'lemma'
    #                     ))
    #     return matches
    
    # def find_synonym_matches(self, entities: List[Dict]) -> List[EntityMatch]:
    #     """Поиск совпадений среди синонимов"""
    #     matches = []
    #     word_to_group = {}
    #     for main_word, synonyms in self.synonyms.items():
    #         word_to_group[main_word] = main_word
    #         for syn in synonyms:
    #             word_to_group[syn] = main_word
    #     group_to_entities = defaultdict(list)
    #     for entity in entities:
    #         normalized_name = self.normalize_text(entity['name'])
    #         for word, group in word_to_group.items():
    #             if word in normalized_name or normalized_name in word:
    #                 group_to_entities[group].append(entity['id'])
    #                 break
    #     for group, entity_ids in group_to_entities.items():
    #         if len(entity_ids) > 1:
    #             for i in range(len(entity_ids)):
    #                 for j in range(i + 1, len(entity_ids)):
    #                     matches.append(EntityMatch(
    #                         entity_ids[i], entity_ids[j], 0.8, 'synonym'
    #                     ))
    #     return matches

    def _generate_matches_from_groups(self, groups: Dict, score: float, match_type: str) -> List[EntityMatch]:
        """Общий метод для генерации совпадений из групп сущностей"""
        matches = []
        for entity_ids in groups.values():
            if len(entity_ids) > 1:
                matches.extend(
                    EntityMatch(id1, id2, score, match_type)
                    for id1, id2 in combinations(entity_ids, 2)
                )
        return matches

    def find_exact_matches(self, entities: List[Dict]) -> List[EntityMatch]:
        """Поиск точных совпадений по названию"""
        name_to_entities = defaultdict(list)
        for entity in entities:
            normalized_name = self.normalize_text(entity['name'])
            name_to_entities[normalized_name].append(entity['id'])
        
        return self._generate_matches_from_groups(groups = name_to_entities, score = 1.0, match_type = 'exact')

    def find_lemma_matches(self, entities: List[Dict]) -> List[EntityMatch]:
        """Поиск совпадений по леммам"""
        if not self.nlp:
            return []
        
        lemma_to_entities = defaultdict(list)
        for entity in entities:
            lemma = self.get_lemma(self.normalize_text(entity['name']))
            if lemma:
                lemma_to_entities[lemma].append(entity['id'])
        
        return self._generate_matches_from_groups(groups = lemma_to_entities, score = 0.9, match_type = 'lemma')

    def find_synonym_matches(self, entities: List[Dict]) -> List[EntityMatch]:
        """Поиск совпадений среди синонимов"""
        word_to_group = {}
        for main_word, synonyms in self.synonyms.items():
            word_to_group[main_word] = main_word
            for syn in synonyms:
                word_to_group[syn] = main_word
        
        group_to_entities = defaultdict(list)
        for entity in entities:
            normalized_name = self.normalize_text(entity['name'])
            matched_group = None
            for word, group in word_to_group.items():
                if word in normalized_name or normalized_name in word:
                    matched_group = group
                    break
            
            if matched_group:
                group_to_entities[matched_group].append(entity['id'])
        
        return self._generate_matches_from_groups(groups = group_to_entities, score = 0.8, match_type= 'synonym')

    def find_semantic_matches(self, entities: List[Dict], threshold: float = 0.7) -> List[EntityMatch]:
        """Поиск семантических совпадений с помощью эмбеддингов"""
        if not self.use_llm or not self.sentence_model:
            return []
        matches = []
        texts = []
        entity_ids = []
        for entity in entities:
            text = f"{entity['name']} {entity.get('description', '')}"
            texts.append(text)
            entity_ids.append(entity['id'])
        if len(texts) < 2:
            return matches
        try:
            embeddings = self.sentence_model.encode(texts)
            similarity_matrix = cosine_similarity(embeddings)
            for i in range(len(entities)):
                for j in range(i + 1, len(entities)):
                    similarity = similarity_matrix[i][j]
                    if similarity >= threshold:
                        matches.append(EntityMatch(
                            entity_ids[i], entity_ids[j], float(similarity), 'semantic'
                        ))
        except Exception as e:
            print(f"Ошибка при семантическом анализе: {e}")
        return matches
    
    def merge_entities(self, entities: List[Dict], relations: List[Dict], 
                      matches: List[EntityMatch]) -> Tuple[List[Dict], List[Dict]]:
        """Объединение энтити и обновление связей"""
        merge_groups = defaultdict(set)
        entity_to_group = {}
        for match in matches:
            group_id = None
            if match.entity1_id in entity_to_group:
                group_id = entity_to_group[match.entity1_id]
            elif match.entity2_id in entity_to_group:
                group_id = entity_to_group[match.entity2_id]
            else:
                group_id = match.entity1_id 
            merge_groups[group_id].add(match.entity1_id)
            merge_groups[group_id].add(match.entity2_id)
            entity_to_group[match.entity1_id] = group_id
            entity_to_group[match.entity2_id] = group_id
        id_mapping = {}
        for group_id, entity_ids in merge_groups.items():
            for entity_id in entity_ids:
                id_mapping[entity_id] = group_id
        merged_entities = []
        processed_groups = set()
        for entity in entities:
            entity_id = entity['id']
            if entity_id in entity_to_group:
                group_id = entity_to_group[entity_id]
                if group_id not in processed_groups:
                    group_entities = [e for e in entities if e['id'] in merge_groups[group_id]]
                    merged_entity = self.create_merged_entity(group_entities, group_id)
                    merged_entities.append(merged_entity)
                    processed_groups.add(group_id)
            else:
                merged_entities.append(entity)
        updated_relations = []
        for relation in relations:
            new_relation = relation.copy()
            if relation['source'] in id_mapping:
                new_relation['source'] = id_mapping[relation['source']]
            if relation['target'] in id_mapping:
                new_relation['target'] = id_mapping[relation['target']]
            if new_relation['source'] != new_relation['target']:
                updated_relations.append(new_relation)
        unique_relations = []
        seen_relations = set()
        for relation in updated_relations:
            relation_key = (relation['source'], relation['target'], relation['type'])
            if relation_key not in seen_relations:
                unique_relations.append(relation)
                seen_relations.add(relation_key)
        return merged_entities, unique_relations
    
    def create_merged_entity(self, entities: List[Dict], new_id: str) -> Dict:
        """Создание объединенной энтити"""
        main_entity = max(entities, key=lambda e: len(e.get('description', '')))
        names = list(set(e['name'] for e in entities))
        descriptions = [e.get('description', '') for e in entities if e.get('description')]
        types = list(set(e.get('type', '') for e in entities if e.get('type')))
        source_chunks = list(set(chunk for e in entities for chunk in e.get('source_chunks', [])))
        merged_entity = {
            'id': new_id,
            'name': main_entity['name'],
            'alternative_names': names[1:] if len(names) > 1 else [],
            'type': types[0] if types else main_entity.get('type', ''),
            'description': '; '.join(filter(None, descriptions)),
            'source_chunks': sorted(source_chunks)
        }
        return merged_entity
    
    async def update_synonims(self, entities : List[Dict]):
        syn = await extract_synonyms_for_entities(entities=entities, graphembedder=self.graphembedder)
        self.synonyms.update(syn)

    async def deduplicate(self, data: Dict, semantic_threshold: float = 0.7) -> Dict:
        """Основная функция дедупликации"""
        entities = data.get('entities', [])
        relations = data.get('relations', [])
        await self.update_synonims(entities)
        all_matches = []
        exact_matches = self.find_exact_matches(entities)
        all_matches.extend(exact_matches)
        lemma_matches = self.find_lemma_matches(entities)
        all_matches.extend(lemma_matches)
        synonym_matches = self.find_synonym_matches(entities)
        all_matches.extend(synonym_matches)
        if self.use_llm:
            semantic_matches = self.find_semantic_matches(entities, semantic_threshold)
            all_matches.extend(semantic_matches)
        unique_matches = []
        seen_pairs = set()
        for match in all_matches:
            pair = tuple(sorted([match.entity1_id, match.entity2_id]))
            if pair not in seen_pairs:
                unique_matches.append(match)
                seen_pairs.add(pair)
        merged_entities, updated_relations = self.merge_entities(entities, relations, unique_matches)
        result = data.copy()
        result['entities'] = merged_entities
        result['relations'] = updated_relations
        if 'stats' in result:
            result['stats']['total_entities'] = len(merged_entities)
            result['stats']['total_relations'] = len(updated_relations)
        return result
    

In [39]:
# Мало связей между энтити и выделенны не все энтити, некоторые сложные понятия не раскрыты
class RDFReasoningEngine:
    """Движок логических рассуждений для графа знаний"""
    
    def __init__(self, knowledge_graph: Dict):
        self.entities = {}
        self.relations = []
        self.adjacency_graph = defaultdict(list)
        self.reverse_graph = defaultdict(list)
        self.type_hierarchy = defaultdict(set)
        
        self._load_knowledge_graph(knowledge_graph)
        self._build_graphs()
        self._infer_type_hierarchy()
    
    def _load_knowledge_graph(self, kg: Dict):
        """Загружает граф знаний из JSON"""

        for entity_data in kg.get('entities', []):
            entity = Entity(
                id=entity_data['id'],
                name=entity_data['name'],
                type=entity_data['type'],
                description=entity_data['description'],
                source_chunks=entity_data.get('source_chunks', [])
            )
            self.entities[entity.name] = entity
        
        for relation_data in kg.get('entities', []):
            if 'source' in relation_data and 'target' in relation_data:
                relation = Relation(
                    source=relation_data['source'],
                    target=relation_data['target'],
                    type=relation_data['type'],
                    description=relation_data['description'],
                    source_chunks=relation_data.get('source_chunks', [])
                )
                self.relations.append(relation)
    
    def _build_graphs(self):
        """Строит графы смежности для быстрого поиска"""
        for relation in self.relations:
            self.adjacency_graph[relation.source].append(relation)
            self.reverse_graph[relation.target].append(relation)
    
    def _infer_type_hierarchy(self):
        """Выводит иерархию типов на основе связей"""
        for entity_name, entity in self.entities.items():
            self.type_hierarchy[entity.type].add(entity_name)
    
    def find_path(self, start: str, end: str, max_depth: int = 5) -> List[List[Relation]]:
        """Находит все пути между двумя сущностями"""
        paths = []
        queue = deque([(start, [])])
        visited = set()
        
        while queue:
            current, path = queue.popleft() 
            if len(path) > max_depth:
                continue
            if current == end and path:
                paths.append(path)
                continue
            state = (current, tuple(r.type for r in path))
            if state in visited:
                continue
            visited.add(state)
            for relation in self.adjacency_graph[current]:
                if relation.target not in [r.target for r in path]: 
                    queue.append((relation.target, path + [relation]))
        return paths
    
    def logical_inference(self, premises: List[Tuple[str, str, str]]) -> List[Dict]:
        """
        Логический вывод на основе предпосылок
        premises: список троек (субъект, предикат, объект)
        """
        inferences = []
        
        for subject, predicate, obj in premises:
            if predicate in ['used-for', 'produces', 'enables', 'controls']:
                transitive_inferences = self._check_transitivity(subject, predicate, obj)
                inferences.extend(transitive_inferences)
            if predicate in ['similar-to', 'co-occurs-with']:
                symmetric_inference = self._check_symmetry(subject, predicate, obj)
                if symmetric_inference:
                    inferences.append(symmetric_inference)
            type_inferences = self._check_type_hierarchy(subject, obj)
            inferences.extend(type_inferences)
        
        return inferences
    
    def _check_transitivity(self, subject: str, predicate: str, obj: str) -> List[Dict]:
        """Проверяет транзитивные отношения A->B, B->C => A->C"""
        inferences = []
        for relation in self.adjacency_graph[obj]:
            if relation.type == predicate:
                inference = {
                    'type': 'transitive',
                    'rule': f'({subject} {predicate} {obj}) ∧ ({obj} {predicate} {relation.target}) → ({subject} {predicate} {relation.target})',
                    'conclusion': (subject, predicate, relation.target),
                    'confidence': 0.8,
                    'evidence': [obj]
                }
                inferences.append(inference)
        
        return inferences
    
    def _check_symmetry(self, subject: str, predicate: str, obj: str) -> Optional[Dict]:
        """Проверяет симметричные отношения A~B => B~A"""
        for relation in self.adjacency_graph[obj]:
            if relation.target == subject and relation.type == predicate:
                return None  
        return {
            'type': 'symmetric',
            'rule': f'({subject} {predicate} {obj}) → ({obj} {predicate} {subject})',
            'conclusion': (obj, predicate, subject),
            'confidence': 0.9,
            'evidence': [subject]
        }
    
    def _check_type_hierarchy(self, subject: str, obj: str) -> List[Dict]:
        """Проверяет отношения на основе иерархии типов"""
        inferences = []
        if subject in self.entities and obj in self.entities:
            subj_type = self.entities[subject].type
            obj_type = self.entities[obj].type
            if subj_type == obj_type:
                inference = {
                    'type': 'type_similarity',
                    'rule': f'type({subject}) = type({obj}) → similar_type({subject}, {obj})',
                    'conclusion': (subject, 'similar_type', obj),
                    'confidence': 0.7,
                    'evidence': [subj_type]
                }
                inferences.append(inference)
        return inferences
    
    def reasoning_chain(self, query: str, context: List[str] = None) -> Dict:
        """
        Цепочка рассуждений для ответа на вопрос
        """
        key_terms = self._extract_key_terms(query)
        relevant_entities = []
        for term in key_terms:
            matches = self._find_similar_entities(term)
            relevant_entities.extend(matches)
        reasoning_steps = []
        if len(relevant_entities) >= 2:
            for i in range(len(relevant_entities)):
                for j in range(i+1, len(relevant_entities)):
                    paths = self.find_path(relevant_entities[i].name, relevant_entities[j].name)
                    if paths:
                        step = {
                            'type': 'connection',
                            'entities': [relevant_entities[i].name, relevant_entities[j].name],
                            'paths': [self._path_to_string(path) for path in paths[:3]],
                            'reasoning': f"Найдены связи между {relevant_entities[i].name} и {relevant_entities[j].name}"
                        }
                        reasoning_steps.append(step)
        premises = []
        for relation in self.relations[:10]: 
            premises.append((relation.source, relation.type, relation.target))
        inferences = self.logical_inference(premises)  
        return {
            'query': query,
            'key_terms': key_terms,
            'relevant_entities': [e.name for e in relevant_entities],
            'reasoning_steps': reasoning_steps,
            'inferences': inferences[:5],  
            'confidence': self._calculate_overall_confidence(reasoning_steps, inferences)
        }
    
    def _extract_key_terms(self, text: str) -> List[str]:
        """Извлекает ключевые термины из текста"""
        words = re.findall(r'\b[а-яё]+\b', text.lower())
        return [word for word in words if len(word) > 3]
    
    def _find_similar_entities(self, term: str) -> List[Entity]:
        """Находит похожие сущности по названию"""
        matches = []
        for entity in self.entities.values():
            if term.lower() in entity.name.lower() or term.lower() in entity.description.lower():
                matches.append(entity)
        return matches
    
    def _path_to_string(self, path: List[Relation]) -> str:
        """Преобразует путь в читаемую строку"""
        if not path:
            return ""
        result = path[0].source
        for relation in path:
            result += f" --[{relation.type}]--> {relation.target}"
        return result
    
    def _calculate_overall_confidence(self, steps: List[Dict], inferences: List[Dict]) -> float:
        """Вычисляет общую уверенность в рассуждении"""
        if not steps and not inferences:
            return 0.0
        total_confidence = 0.0
        count = 0
        
        for inference in inferences:
            total_confidence += inference.get('confidence', 0.5)
            count += 1
        total_confidence += len(steps) * 0.3
        count += len(steps)
        return min(total_confidence / max(count, 1), 1.0)
    
    def validate_knowledge_consistency(self) -> Dict:
        """Проверяет консистентность базы знаний"""
        issues = []
        cycles = self._detect_cycles()
        if cycles:
            issues.append({
                'type': 'circular_dependency',
                'description': 'Обнаружены циклические зависимости',
                'details': cycles
            })
        contradictions = self._detect_contradictions()
        if contradictions:
            issues.append({
                'type': 'contradiction',
                'description': 'Обнаружены противоречия',
                'details': contradictions
            })
        return {
            'is_consistent': len(issues) == 0,
            'issues': issues,
            'statistics': {
                'entities_count': len(self.entities),
                'relations_count': len(self.relations),
                'types_count': len(self.type_hierarchy)
            }
        }
    
    def _detect_cycles(self) -> List[List[str]]:
        """Обнаруживает циклы в графе"""
        visited = set()
        rec_stack = set()
        cycles = []
        def dfs(node, path):
            if node in rec_stack:
                cycle_start = path.index(node)
                cycles.append(path[cycle_start:] + [node])
                return
            if node in visited:
                return
            visited.add(node)
            rec_stack.add(node)
            for relation in self.adjacency_graph[node]:
                dfs(relation.target, path + [node])
            rec_stack.remove(node)
        for entity_name in self.entities.keys():
            if entity_name not in visited:
                dfs(entity_name, [])
        return cycles
    
    def _detect_contradictions(self) -> List[Dict]:
        """Обнаруживает логические противоречия"""
        contradictions = []
        for relation1 in self.relations:
            for relation2 in self.relations:
                if (relation1.source == relation2.target and 
                    relation1.target == relation2.source and
                    self._are_contradictory(relation1.type, relation2.type)):
                    contradictions.append({
                        'type': 'bidirectional_contradiction',
                        'relation1': f"{relation1.source} {relation1.type} {relation1.target}",
                        'relation2': f"{relation2.source} {relation2.type} {relation2.target}"
                    })
        return contradictions
    
    def _are_contradictory(self, type1: str, type2: str) -> bool:
        """Проверяет, являются ли два типа отношений противоречивыми"""
        contradictory_pairs = {
            ('enables', 'prevents'),
            ('produces', 'destroys'),
            ('supports', 'opposes')
        }
        return (type1, type2) in contradictory_pairs or (type2, type1) in contradictory_pairs

In [40]:

def transform_knowledge_graph_to_gephi(json_data: Dict[str, Any], 
                                     nodes_output_path: str = "C:/Users/user/Desktop/nodes.csv",
                                     edges_output_path: str = "C:/Users/user/Desktop/edges.csv") -> None:
    """
    Трансформирует данные графа знаний из JSON в CSV формат для Gephi.
    
    Args:
        json_data: Словарь с данными графа знаний
        nodes_output_path: Путь для сохранения файла узлов
        edges_output_path: Путь для сохранения файла рёбер
    """

    nodes_data = []
    
    for entity in json_data.get("entities", []):
        node = {
            "Id": entity["id"],
            "Label": entity["name"],
            "Type": entity.get("type", "unknown"),
            "Description": entity.get("description", "").replace("\n", " ").replace(";", ","),
            "Alternative_Names": "|".join(entity.get("alternative_names", [])),
            "Source_Chunks": "|".join(map(str, entity.get("source_chunks", [])))
        }
        nodes_data.append(node)
    
    nodes_df = pd.DataFrame(nodes_data)
    nodes_df.to_csv(nodes_output_path, index=False, encoding='utf-8-sig')
    
    edges_data = []
    edge_id = 0
    
    for relation in json_data.get("relations", []):
        edge = {
            "Id": edge_id,
            "Source": relation["source"],
            "Target": relation["target"],
            "Type": relation.get("type", "unknown"),
            "Label": relation.get("type", "unknown"),
            "Description": relation.get("description", "").replace("\n", " ").replace(";", ","),
            "Source_Chunks": "|".join(map(str, relation.get("source_chunks", [])))
        }
        edges_data.append(edge)
        edge_id += 1

    edges_df = pd.DataFrame(edges_data)
    edges_df.to_csv(edges_output_path, index=False, encoding='utf-8-sig')
    
    print(f"Создано узлов: {len(nodes_data)}")
    print(f"Создано рёбер: {len(edges_data)}")
    print(f"Файлы сохранены: {nodes_output_path}, {edges_output_path}")


def load_and_transform(json_file_path: str) -> None:
    """
    Загружает JSON файл и трансформирует его в CSV для Gephi.
    
    Args:
        json_file_path: Путь к JSON файлу с данными графа знаний
    """
    try:
        with open(json_file_path, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
        
        transform_knowledge_graph_to_gephi(json_data)
        
    except FileNotFoundError:
        print(f"Файл {json_file_path} не найден")
    except json.JSONDecodeError:
        print(f"Ошибка при парсинге JSON файла {json_file_path}")
    except Exception as e:
        print(f"Произошла ошибка: {e}")


def transform_from_string(json_string: str) -> None:
    """
    Трансформирует JSON строку в CSV для Gephi.
    
    Args:
        json_string: JSON строка с данными графа знаний
    """
    try:
        json_data = json.loads(json_string)
        transform_knowledge_graph_to_gephi(json_data)
        
    except json.JSONDecodeError:
        print("Ошибка при парсинге JSON строки")
    except Exception as e:
        print(f"Произошла ошибка: {e}")

        

In [ ]:
async def create_knowledge_graph(text_path, graph_path):
    gemini_api_keys=[

                    ]
    generation_model_path = "C:/Users/user/AppData/Local/nomic.ai/GPT4All/qwen2.5-coder-7b-instruct-q4_0.gguf" 
    processor = GeminiEnhancedTextProcessor(gemini_api_keys)
    graphembedder = ImprovedKnowledgeExtractor(gemini_api_keys, generation_model_path)
    deduplicator = EntityDeduplicator(graphembedder)
    with open(text_path, encoding='utf-8') as f:
        text = f.read()
    result = await processor.process_text_enhanced(text)
    graph_data = await graphembedder.extract_knowledge_graph(result['processed_text'])
    unique_graph_data = await deduplicator.deduplicate(graph_data, semantic_threshold=0.7)
    with open(graph_path, "w", encoding='utf-8') as fl:
        json.dump(unique_graph_data, fl, ensure_ascii=False, indent=4)
    return unique_graph_data

In [43]:
def reasoning_system(query: str, graph_path = 'C:/Users/user/Desktop/fullgraphdata.json'):
    
    with open(graph_path, 'r', encoding='utf-8') as f:
        graph = json.load(f)

    reasoning_engine = RDFReasoningEngine(graph)

    result = reasoning_engine.reasoning_chain(query)
    
    print(f"Запрос: {result['query']}")
    print(f"Ключевые термины: {result['key_terms']}")
    print(f"Релевантные сущности: {result['relevant_entities']}")
    print(f"Уверенность: {result['confidence']:.2f}")
    
    if result['reasoning_steps']:
        print("\nШаги рассуждения:")
        for i, step in enumerate(result['reasoning_steps'], 1):
            print(f"{i}. {step['reasoning']}")
    
    if result['inferences']:
        print("\nЛогические выводы:")
        for i, inference in enumerate(result['inferences'], 1):
            print(f"{i}. {inference['rule']} (уверенность: {inference['confidence']})")
    
    consistency = reasoning_engine.validate_knowledge_consistency()
    print(f"\nКонсистентность базы знаний: {'✓' if consistency['is_consistent'] else '✗'}")
    print(f"Статистика: {consistency['statistics']}")

In [ ]:
async def main():
    unique_graph_data = await create_knowledge_graph('C:/Users/user/Desktop/rawmaterial.txt', 'C:/Users/user/Desktop/fullgraphdata.json')
    await asyncio.sleep(1)
    print(unique_graph_data)
    transform_knowledge_graph_to_gephi(unique_graph_data)
await main()